In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
import nltk
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from sklearn.decomposition import LatentDirichletAllocation as LDA
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from gensim import corpora, models
from pprint import pprint

from dfunc import lemmatize
from dfunc import preprocess

sns.set_style('whitegrid')
%matplotlib inline

In [2]:
# Set DataFrame to only be 'descriptions' and 'fraudulent'
df = pd.read_csv('fake_job_postings.csv')
df = df.loc[df['location'].str[:2] == 'US']
df = df[['description', 'fraudulent']]

In [16]:
# Preprocess 'description' feature and tokenize
processed_des = df['description'].map(preprocess)

In [21]:
# Generate dictionary from tokens
dictionary = gensim.corpora.Dictionary(processed_des)

In [23]:
# Filter out extremely common and uncommon words
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [29]:
# Create bag of words for each description
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_des]

In [46]:
# Instantiate TfidfModel and transform bag of words
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [53]:
# Create LDAMulticore model with bag of words
lda_model = gensim.models.LdaMulticore(bow_corpus,
                                       num_topics=2,
                                       id2word=dictionary,
                                       passes=2,
                                       workers=2)